#### 导入数据

In [54]:
import pandas as pd
import numpy as np

#reading the dataset
df=pd.read_csv("E:/2018/Quantitative/abuQuant/myproject/ML/all-data/load_prediction/train_u6lujuX_CVtuZ9i.csv")

#filling missing values
df['Gender'].fillna('Male', inplace=True)
df = df.dropna()
df = df.drop('Loan_ID',axis=1)
print(df.head(), np.shape(df))
#数据转换：将字符型的变量数值化
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in ['Self_Employed', 'Gender','Married','Education','Property_Area','Loan_Status']:
    df[i] = le.fit_transform(df[i])
df['Dependents'] = df['Dependents'].apply(lambda x: 3 if len(str(x))>1 else x)
for col in df.columns[1:-1]:
    print(col)
    df[col] = df[col].apply(lambda x:float(x))
print(df.head(), np.shape(df))

  Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
1   Male     Yes          1      Graduate            No             4583   
2   Male     Yes          0      Graduate           Yes             3000   
3   Male     Yes          0  Not Graduate            No             2583   
4   Male      No          0      Graduate            No             6000   
5   Male     Yes          2      Graduate           Yes             5417   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1             1508.0       128.0             360.0             1.0   
2                0.0        66.0             360.0             1.0   
3             2358.0       120.0             360.0             1.0   
4                0.0       141.0             360.0             1.0   
5             4196.0       267.0             360.0             1.0   

  Property_Area Loan_Status  
1         Rural           N  
2         Urban           Y  
3         Urban           Y  
4 

#### 切分数据

In [55]:
#split dataset into train and test

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=0)

x_train=train.drop('Loan_Status',axis=1)
y_train=train['Loan_Status']

x_test=test.drop('Loan_Status',axis=1)
y_test=test['Loan_Status']

print(np.shape(x_train), np.shape(x_test))
#create dummies
x_trainD=pd.get_dummies(x_train)
x_testD=pd.get_dummies(x_test)

(344, 11) (148, 11)


### bagging
#### 1、bagging meta-estimator

In [56]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
model.fit(x_train, y_train)
print(model.predict(x_test))
model.score(x_test,y_test)

[1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 0 0 1 1
 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0
 1 0 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1]


0.722972972972973

In [57]:
from sklearn.ensemble import BaggingRegressor
model = BaggingRegressor(tree.DecisionTreeRegressor(random_state=1))
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.1345104895104896

#### 2、random forest

In [58]:
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.75

In [59]:
#feature importance
for i, j in sorted(zip(x_train.columns, model.feature_importances_)):
    print(i, j)

ApplicantIncome 0.21363977426486325
CoapplicantIncome 0.1090427447163614
Credit_History 0.22413033268854354
Dependents 0.04775049741183001
Education 0.026898772034271067
Gender 0.02837953061157672
LoanAmount 0.19431967245420506
Loan_Amount_Term 0.046621032038821744
Married 0.028931727663536043
Property_Area 0.05904900872431605
Self_Employed 0.021236907391675153


In [60]:
from sklearn.ensemble import RandomForestRegressor
model= RandomForestRegressor()
model.fit(x_train, y_train)
model.score(x_test,y_test)

D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.13321678321678332

### boosting
#### 1、AdaBoost

In [61]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.75

In [62]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.13435923721306975

#### 2、GBM

In [63]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.8108108108108109

In [65]:
#### 3、XGB
#分类
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(x_train, y_train)
model.score(x_test,y_test)

#回归
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(x_train, y_train)
model.score(x_test,y_test)

ModuleNotFoundError: No module named 'xgboost'

#### 4、Light GBM

In [ ]:
#分类
import lightgbm as lgb
train_data=lgb.Dataset(x_train,label=y_train)
#define parameters
params = {'learning_rate':0.001}
model= lgb.train(params, train_data, 100) 
y_pred=model.predict(x_test)
for i in range(0,185):
   if y_pred[i]>=0.5: 
   y_pred[i]=1
else: 
   y_pred[i]=0

#回归
import lightgbm as lgb
train_data=lgb.Dataset(x_train,label=y_train)
params = {'learning_rate':0.001}
model= lgb.train(params, train_data, 100)
from sklearn.metrics import mean_squared_error
rmse=mean_squared_error(y_pred,y_test)**0.5